### RNN on MNIST

In [2]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
# data
mnist_train = torchvision.datasets.MNIST(root="./data", train=True, transform=torchvision.transforms.ToTensor(),download=True)
mnist_test = torchvision.datasets.MNIST(root="./data", train=False, transform=torchvision.transforms.ToTensor(),download=True)

Processing...
Done!


In [4]:
# hyperparameters
seq_len = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 1e-3

In [5]:
# data loader
train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size, shuffle=True)

In [11]:
class RNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, batch_first = True)
        self.fc = torch.nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [12]:
model = RNN(input_size, hidden_size, num_classes, num_layers).to(device)

In [13]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, seq_len, input_size).to(device)
        labels = labels.to(device)
        
        # Forward
        output = model(images)
        loss = loss_fn(output, labels)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print ("Epoch: ",epoch)

Epoch:  0
Epoch:  1


In [ ]:
# test
with torch.no_grad():
    for images, labels in test_loader:
        